In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [10]:
df_final=pd.read_csv('../data/processed/data_limpio.csv')

In [11]:
# Supongamos que X e y son tus datos completos (combined_text y label)
X = df_final['combined_text'].values
y = df_final['label'].values

# PASO 1: Separar el TEST definitivo (digamos, 20%)
# X_temp contiene el 80% restante (que será Train + Val)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PASO 2: Separar TRAIN y VALIDATION de ese X_temp
# Queremos que Val sea mas o menos el 20% del total original.
# Si X_temp es el 80% del total, sacar el 25% de X_temp nos da el 20% del total (0.8 * 0.25 = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f"Total original: {len(X)}")
print(f"Training set (60%):   {len(X_train)}")
print(f"Validation set (20%): {len(X_val)}")
print(f"Test set (20%):       {len(X_test)}")

Total original: 60994
Training set (60%):   36596
Validation set (20%): 12199
Test set (20%):       12199


In [12]:
# Hiperparámetros
VOCAB_SIZE = 10000 # Tamaño del vocabulario
MAX_LENGTH = 250 # Longitud máxima de las secuencias
OOV_TOK = "<OOV>" # Token para palabras fuera del vocabulario

# 1. Tokenizer (SOLO con Train)
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOK)
tokenizer.fit_on_texts(X_train) 

# 2. Convertir a secuencias
train_sequences = tokenizer.texts_to_sequences(X_train)
val_sequences = tokenizer.texts_to_sequences(X_val)   # Usamos el tokenizer ya entrenado
test_sequences = tokenizer.texts_to_sequences(X_test) # Usamos el tokenizer ya entrenado

# 3. Padding
X_train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_val_padded = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(test_sequences, maxlen=MAX_LENGTH, padding='post', truncating='post')

# Convertir a numpy arrays
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [13]:
# Dimensión de los vectores de embedding (16, 32, 64 son comunes)
EMBEDDING_DIM = 16 

model = tf.keras.Sequential([
    # Capa 1: Embedding (Aprende relaciones entre palabras)
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    
    # Capa 2: LSTM (Bidireccional es mejor porque lee izquierda-derecha y viceversa)
    # Dropout ayuda a evitar el Overfitting (memorización)
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
    
    # Capa 3: Densa para procesar lo aprendido
    tf.keras.layers.Dense(24, activation='relu'),
    
    # Capa 4: Dropout extra de seguridad
    tf.keras.layers.Dropout(0.5),
    
    # Capa de Salida: 1 sola neurona con Sigmoid (porque la salida es 0 o 1)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilación
model.compile(loss='binary_crossentropy', # Función de pérdida para clasificación binaria
              optimizer='adam',           # El mejor optimizador general
              metrics=['accuracy'])

model.summary()

/home/rodrick/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-12-13 00:39:00.511498: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Asumiendo que ya definiste el modelo 'model' como hicimos antes...

history = model.fit(
    X_train_padded, y_train, 
    epochs=5,
    # AQUI está la clave: usamos validation_data con el set de validación
    validation_data=(X_val_padded, y_val), 
    verbose=1
)

# --- EVALUACIÓN FINAL ---
# Solo ahora, al final de todo, miramos el Test Set
print("\n--- Evaluación Final en Test Set ---")
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Precisión Real del Modelo: {accuracy*100:.2f}%")

Epoch 1/5
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 107s 91ms/step - accuracy: 0.8645 - loss: 0.2976 - val_accuracy: 0.9598 - val_loss: 0.1120
Epoch 2/5
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 105s 92ms/step - accuracy: 0.9714 - loss: 0.0888 - val_accuracy: 0.9660 - val_loss: 0.1003
Epoch 3/5
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 100s 88ms/step - accuracy: 0.9819 - loss: 0.0570 - val_accuracy: 0.9622 - val_loss: 0.1140
Epoch 4/5
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 98s 86ms/step - accuracy: 0.9921 - loss: 0.0302 - val_accuracy: 0.9628 - val_loss: 0.1258
Epoch 5/5
1144/1144 ━━━━━━━━━━━━━━━━━━━━ 99s 87ms/step - accuracy: 0.9942 - loss: 0.0210 - val_accuracy: 0.9650 - val_loss: 0.1711

--- Evaluación Final en Test Set ---
382/382 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.9649 - loss: 0.1587
Precisión Real del Modelo: 96.40%


In [15]:
import pickle

# 1. Guardar el Modelo (formato nativo de Keras)
model.save('detector_fake_news.keras')
print("Modelo guardado como 'detector_fake_news.keras'")

# 2. Guardar el Tokenizer (usando Pickle)
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Tokenizer guardado como 'tokenizer.pickle'")

Modelo guardado como 'detector_fake_news.keras'
Tokenizer guardado como 'tokenizer.pickle'
